works with kernel Python \[conda env:pangu] as constructed in README

run fengwu with 1 GPU and 20 GB memory (must be v100, not gp100 because gp100 only has 16 GB VRAM, but v100 has 32GB)

Make these the default casper modules `module save`

```
Currently Loaded Modules:
  1) ncarenv/24.12  (S)   3) ncarcompilers/1.0.0   5) ucx/1.17.0      7) hdf5/1.12.3    9) cudnn/9.2.0.82-12
  2) intel/2024.2.1       4) cuda/12.3.2           6) openmpi/5.0.6   8) netcdf/4.9.2  10) conda/latest
```

Inferences in conda env:ainwp is different by 0.0001 K from pangu env (after 240 hours)
conda env:ainwp is supposed to replicate realtime runs, but still 0.1 K different from realtime runs

In [1]:
import argparse
import os
import subprocess
from pathlib import Path

import earthkit.data as ekd
import pandas as pd
import xarray
from run_pangu import plot_ensemble
from run_pangu.s3_run_fengwu_ecmwf import (
    pressure_levels,
    variables,
)

ai_models_dir = Path("/glade/derecho/scratch/ahijevyc/ai-models")
date = pd.to_datetime("2024042500", format="%Y%m%d%H")
ic = "gefs"  # "gefs" (GFS Ensemble Forecast System) or "ens" (ECMWF's Ensemble Prediction System)
fhr_end = 24

In [2]:
!uname -a

Linux crhtc73 5.14.21-150400.24.46-default #1 SMP PREEMPT_DYNAMIC Thu Feb 9 08:38:18 UTC 2023 (2d95137) x86_64 x86_64 x86_64 GNU/Linux


In [3]:
def ai_input_nc(nc):
    input = xarray.open_dataset(nc)
    sfc_param = ["u10m", "v10m", "t2m", "msl"]
    pl_param = [f"{f}{p}" for f in variables for p in pressure_levels]
    fields_all = []
    for p in sfc_param + pl_param:
        field = input["__xarray_dataarray_variable__"].sel(channel=p).squeeze().values
        fields_all.append(field)
    return np.stack(fields_all)

In [ ]:
if ic == "gefs":
    mems = ["c00"] + [f"p{m:02d}" for m in range(1, 31)]
    files = [
        ai_models_dir
        / "input"
        / date.strftime("%Y%m%d%H")
        / mem
        / f"ge{mem}.t{date:%H}z.pgrb.0p25.f000"
        for mem in mems
    ]
    opfx = "ge"
elif ic == "ens":
    assert date > pd.to_datetime(
        "20250209"
    ), "started saving ECMWF ensemble forecast (ens) after 20250209"
    mems = [f"ens{m}" for m in range(0, 51)]
    # 2025 realtime input in Ryan's scratch directory
    files = [
        f"/glade/derecho/scratch/sobash/pangu_realtime/{date:%Y%m%d%H}/{mem}/{mem}_analysis_{date:%Y%m%d%H}.grib2"
        for mem in mems
    ]
    opfx = ""

for mem, file in zip(mems, files):
    odir = ai_models_dir / f"output/panguweather/{date:%Y%m%d%H}"
    path = odir / f"{opfx}{mem}.grib"
    os.makedirs(odir, exist_ok=True)
    if os.path.exists(path):
        continue
    command = [
        "ai-models",
        "panguweather",
        "--input=file",
        f"--file={file}",
        f"--assets={ai_models_dir}",
        "--output=file",
        f"--path={path}",
        f"--lead-time={fhr_end}",
    ]

    print("Executing command:")
    # Use a single space to join list elements for printing
    print(" ".join(command))

    # 3. Run the command
    try:
        # Use subprocess.run to execute the command.
        # check=True will raise an exception if the command fails.
        # capture_output=True and text=True will capture stdout/stderr as text.
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print("STDOUT:", result.stdout)
        print(f"SUCCESS: Forecast for {date.strftime('%Y-%m-%d')} completed.")

    except subprocess.CalledProcessError as e:
        # This block will run if the command returns a non-zero exit code (i.e., fails)
        print(f"ERROR: The command for {date.strftime('%Y-%m-%d')} failed.")
        print("Return Code:", e.returncode)
        print("STDOUT:", e.stdout)
        print("STDERR:", e.stderr)
        # Use 'break' if you want the loop to stop after the first failure
        break

    print("-" * 40)

In [ ]:
args = argparse.Namespace
args.ic = ic.upper()
args.model = "panguweather"
ifiles = (ai_models_dir / f"output/{args.model}/{date:%Y%m%d%H}").glob(f"{opfx}*[0-9].grib")
ifiles = sorted(list(ifiles))
print(len(ifiles))
da = (
    xarray.concat(
        [
            ekd.from_source_lazily("file", ifile)
            .sel(shortName="z", level=500)
            .to_xarray(engine="cfgrib")
            # adapt longitude slice for 0-360 or -180-180
            .assign_coords(longitude=lambda ds: ds.longitude % 360)
            .sel(latitude=slice(60, 20), longitude=slice(220, 300))
            for ifile in ifiles
        ],
        dim="number",
    )
    .squeeze()
    .rename(time="init_time", number="mem")
)
da

In [ ]:
fig = plot_ensemble.plot_forecast_grid(args, da.sortby("mem"), plotdays=[1, 2, 3, 4, 5, 6, 7, 8])